In [1]:
import pandas as pd
import numpy as np
import re 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import  accuracy_score, f1_score

In [2]:
# Read dataset
df = pd.read_csv("electronics_data.csv")


In [3]:
# Pull weight float values from string
df["cleaned_weight"] = df['weight'].apply(lambda x:  re.findall(r'[\d\.\d]+', x) )

In [4]:
# Only keep weight values that have 4 numbers. 
# There are a bunch of rows that have multiple weight values, ignore those
weight_filtered = df[df["cleaned_weight"].apply(lambda x: len(x) ==1)]
weight_filtered["cleaned_weight"] = weight_filtered["cleaned_weight"].apply(lambda x : x[0])

# Keep only the specified columns, just trying to simply  
column_filtered  = weight_filtered[["prices.amountMax","cleaned_weight", "prices.condition", "brand", "categories"]]

<ipython-input-4-1d3e7fc71ce2>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weight_filtered["cleaned_weight"] = weight_filtered["cleaned_weight"].apply(lambda x : x[0])


In [5]:
def clean_condition(row): 
    if "new" in row.lower(): 
        row = "new"
    if "refurbished" in row.lower():
        row = "refurbished"
    return row 

def clean_category(row):
    
    if "stereo" in row.lower() or "audio" in row.lower():
        row = "audio"
    elif "tv" in row.lower() or "television" in row.lower():
        row = "tv"
    elif "camera" in row.lower():
        row = "camera"
    elif "car" in row.lower(): 
        row = "car"
    elif "tablet" in row.lower():
        row = "tablet"
    elif "speakers" in row.lower():
        row ="speakers"
    elif "computer" in row.lower() or "laptops" in row.lower() :
        row ="computer"
    elif "bridges" in row.lower():
        row = "bridges"
    elif "ipod" in row.lower() and "accessories" not in row.lower():
        row = "ipod"
    elif "A/V" in row or "AV" in row: 
        row = "AV related"
    return row

# Cleaning price.condition column 
column_filtered["cleaned_condition"] = column_filtered["prices.condition"].apply(lambda x : clean_condition(x))
# Clean the category column, do not use the categories columns for anything further
column_filtered["cleaned_cats"] = column_filtered["categories"].apply(lambda x: x.split(",")[-1])
column_filtered['cleaned_cats2'] = column_filtered["cleaned_cats"].apply(lambda x: clean_category(x))


<ipython-input-5-c82db012490e>:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  column_filtered["cleaned_condition"] = column_filtered["prices.condition"].apply(lambda x : clean_condition(x))
<ipython-input-5-c82db012490e>:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  column_filtered["cleaned_cats"] = column_filtered["categories"].apply(lambda x: x.split(",")[-1])
<ipython-input-5-c82db012490e>:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .lo

In [6]:

# Create label encoding 
le1 = LabelEncoder()
le2 = LabelEncoder()
le3 = LabelEncoder()

column_filtered['encoded_brand']= le1.fit_transform(column_filtered['brand'])
column_filtered['encoded_condition'] = le2.fit_transform(column_filtered['cleaned_condition'])
column_filtered['encoded_cats'] = le3.fit_transform(column_filtered['cleaned_cats2'])

column_all = column_filtered.drop(columns=["brand", 
                                           "cleaned_condition",
                                           "categories",
                                           "prices.condition",
                                           "cleaned_cats2",
                                           "cleaned_cats"])

<ipython-input-6-c1023e6b0cce>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  column_filtered['encoded_brand']= le1.fit_transform(column_filtered['brand'])
<ipython-input-6-c1023e6b0cce>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  column_filtered['encoded_condition'] = le2.fit_transform(column_filtered['cleaned_condition'])
<ipython-input-6-c1023e6b0cce>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inste

In [7]:
X = column_all.drop(columns=["encoded_brand"])
y = column_all["encoded_brand"]

In [8]:

x_train_val, x_test, y_train_val, y_test = train_test_split(X,y,test_size = 0.10, random_state=1)
x_train, x_val, y_train, y_val = train_test_split(x_train_val, y_train_val, test_size = 0.20, random_state=1)


In [9]:
x_train

,prices.amountMax,cleaned_weight,encoded_condition,encoded_cats
5284,248.00,5.9,1,204
5659,479.99,12.35,1,79
6309,1399.99,5.07,1,196
7052,99.99,4.2,1,202
6340,349.99,8.6,1,192
...,...,...,...,...
4273,1798.00,63.7,1,204
4387,22.06,4.34,1,87
3354,193.93,3.7,1,194
5557,94.99,19.8,1,196


In [10]:
rfc = RandomForestClassifier()
rfc.fit(x_train, y_train)
pred_val = rfc.predict(x_val)


In [11]:
accuracy_score(pred_val, y_val), f1_score(pred_val, y_val, average="weighted")

(0.950168918918919, 0.9535023435583719)